<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_MathChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Using MathChat to Solve Math Problems

In [13]:
!pip install pyautogen[mathchat] --user


  Using cached wolframalpha-5.0.0-py3-none-any.whl (7.5 kB)
  Using cached jaraco.context-4.3.0-py3-none-any.whl (5.3 kB)


In [3]:
!pip install pyautogen==0.2.3

  Using cached pyautogen-0.2.3-py3-none-any.whl (133 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
     -------------------------------------- 257.5/257.5 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 296.7/296.7 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.0
    Uninstalling openai-0.27.0:
      Successfully uninstalled openai-0.27.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
promptify 2.0.3 requires huggingface-hub==0.12, but you have huggingface-hub 0.20.2 which is incompatible.
promptify 2.0.3 requires jinja2==2.11.3, but you have jinja2 3.1.3 which is incompatible.
promptify 2.0.3 requires openai==0.27.0, but you have openai 1.14.0 which is incompatible.


In [17]:
import os

# client = openai.OpenAI()
import autogen
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

config_list = [
    {
        'model': 'gpt-3.5-turbo',
        'api_key': 'your_api',
    }]

## Construct agents for MathChat

We start by initializing the `AssistantAgent` and `MathUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for MathChat. The detailed instructions are given in the user message. 


In [18]:
# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

### Example 1

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

Correct Solution: 
We have \begin{align*} (2x+10)(x+3)&<(3x+9)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)&<3(x+3)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)-3(x+3)(x+8)&<0 \quad \Rightarrow
\\ (2x+10-(3x+24))(x+3)&<0 \quad \Rightarrow
\\ (-x-14)(x+3)&<0 \quad \Rightarrow
\\ (x+14)(x+3)&>0.
\end{align*} This inequality is satisfied if and only if $(x+14)$ and $(x+3)$ are either both positive or both negative.  Both factors are positive for $x>-3$ and both factors are negative for $x<-14$.  When $-14<x<-3$, one factor is positive and the other negative, so their product is negative.   Therefore, the range of $x$ that satisfies the inequality is $ \boxed{(-\infty, -14)\cup(-3,\infty)} $.

In [21]:
# given a math problem, we use the mathproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the mathproxyagent for processing.
# The conversation continues until the termination condition is met, in MathChat, the termination condition is the detect of "\boxed{}" in the response.
math_problem = (
    "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
)

# We call `initiate_chat` to start the conversation.
# When setting `message=mathproxyagent.message_generator`, you need to pass in the problem through the `problem` parameter.
mathproxyagent.initiate_chat(assistant, problem=math_problem)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continu

SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

[-14, -3]

Union(Interval.open(-oo, -14), Interval.open(-3, oo))

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

Based on the calculated roots and the intervals, the solution to the inequality $(2x+10)(x+3)<(3x+9)(x+8)$ is $x \in \boxed{(-\infty, -14) \cup (-3, \infty)}$ in interval notation.

--------------------------------------------------------------------------------


### Example 2
Problem: For what negative value of $k$ is there exactly one solution to the system of equations \begin{align*}
y &= 2x^2 + kx + 6 \\
y &= -x + 4?
\end{align*}

Correct Solution: Setting the two expressions for $y$ equal to each other, it follows that $2x^2 + kx + 6 = -x + 4$. Re-arranging, $2x^2 + (k+1)x + 2 = 0$. For there to be exactly one solution for $x$, then the discriminant of the given quadratic must be equal to zero. Thus, $(k+1)^2 - 4 \cdot 2 \cdot 2 = (k+1)^2 - 16 = 0$, so $k+1 = \pm 4$. Taking the negative value, $k = \boxed{-5}$.

In [22]:
math_problem = "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
mathproxyagent.initiate_chat(assistant, problem=math_problem)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continu

SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

The solutions of the equation are: [-5, 3]

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

Now we have the two solutions for the equation $(k + 1)^2 - 4 \cdot 2 \cdot 2 = 0$, which are $k = -5$ and $k = 3$. The problem however requires us to find what negative value of $k$ is there exactly one solution to the system of equations.

So the answer to the problem is $k=-5$.

Therefore, the negative value of $k$ for which there is exactly one solution to the given system of equations is $\boxed{-5}$.

--------------------------------------------------------------------------------


### Example 3
Problem: Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas.

Correct Solution: For the roots to be real and rational, the discriminant must be a perfect square. Therefore, $(-7)^2-4 \cdot 1 \cdot c = 49-4c$ must be a perfect square. The only positive perfect squares less than 49 are $1$, $4$, $9$, $16$, $25$, and $36$. The perfect squares that give a integer value of $c$ are $1$, $9$, and $25$. Thus, we have the equations $49-4c=1$, $49-4c=9$, and $49-4c=25$. Solving, we get that the positive integer values of c are $\boxed{12, 10, 6}$.

In [24]:
math_problem = "Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas."
mathproxyagent.initiate_chat(assistant, problem=math_problem)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

First state the key idea to solve the problem. You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem is mostly reasoning, you can solve it by yourself directly.
Case 3: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continu

SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

Error:   File "", line 16
    c_values_rational.sort(reverse=True)
    ^
SyntaxError: invalid syntax

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

I apologize for the oversight. There's a small issue with the syntax in line 10 of the provided Python code. I missed the closing bracket on line 10. Here's the corrected version:

```python
import sympy as sp

# define the variable
c = sp.symbols('c')

# discriminant formula
discriminant = (-7)**2 - 4*1*c

# find values where discriminant is greater than or equal to zero
c_values = sp.solve(discriminant >= 0, c)

# check if discriminant for each c is a perfect square
c_values_rational = [val for val in c_values if (sp.sqrt((-7)**2 - 4*1*val)**2 == (-7)**2 - 4*1*val)]

# print the values in decreasing order
c_values_rational.sort(reverse=True)
print('[c] = ',  c_values_rational)
```

This Python code should now run correctly. Please give it a 

SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

Error: Traceback (most recent call last):
  File "", line 15, in <module>
    c_values_rational = [val for val in c_values if (sp.sqrt((-7)**2 - 4*1*val)**2 == (-7)**2 - 4*1*val)]
TypeError: 'And' object is not iterable

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

I apologize for the mistake. Indeed, we receive an error because `c_values` is an "And" object which isn't iterable. This is caused by a misunderstanding of the `solve` function's output. `solve` returns an "And" object that represents multiple equality constraints when we ask it for values that satisfy `b^2 - 4ac >= 0`. Instead, we need to solve for the actual interval of `c` and then iterate over the integer values within this interval. Because the roots are rational, `b^2 - 4ac` must be a perfect square. 

Let's correct the code as follows:

```python
import sympy as sp
import math

# define the variable
c = sp.symbols('c')

...

mathproxyagent (to assistant):

[c] =  []

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

I apologize for the oversight. The range of `m` was not specified before solving for `c`. Therefore, all positive integer values of `m` from 0 to `b` (7 in this case) must be evaluated.

Let's correct the script:

```python
import sympy as sp

# define the variable
c = sp.Symbol('c')
a, b = 1, 7  # define additional symbols

# potential c values for m from 0 to 7 (inclusive)
c_values_rational = [(b**2 - m**2) / (4*a) for m in range(0, b+1)]

# keep only the positive integer values
c_values_rational = [val for val in c_values_rational if val.is_integer() and val > 0]

# print the values in decreasing order
c_values_rational.sort(reverse=True)
print('[c] = ',  c_values_rational)
```

Go ahead and run the corrected Python code, please.

--------------------------------------------------------------------------------


SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

[c] =  [12.0, 10.0, 6.0]

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

I apologize for the error. The list elements are of float type, not integers. Let's convert them to integers and update the script as follows:

```python
import sympy as sp

# define the variable
c = sp.Symbol('c')
a, b = 1, 7  # define additional symbols

# potential c values for m from 0 to 7 (inclusive)
c_values_rational = [(b**2 - m**2) / (4*a) for m in range(0, b+1)]

# keep only the positive integer values
c_values_rational = [int(val) for val in c_values_rational if val.is_integer() and val > 0]

# print values in decreasing order
c_values_rational.sort(reverse=True)
print('[c] = ',  c_values_rational)
```
Then, run the corrected Python code.

--------------------------------------------------------------------------------


SIGALRM is not supported on Windows. No timeout will be enforced.
SIGALRM is not supported on Windows. No timeout will be enforced.


mathproxyagent (to assistant):

[c] =  [12, 10, 6]

--------------------------------------------------------------------------------
assistant (to mathproxyagent):

Great! So, the positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational are $12, 10, 6$.

Hence, the final answer is $\boxed{12, 10, 6}$.

--------------------------------------------------------------------------------


------------------------------------
### Using other prompts

MathChat allows different prompts that instruct the assistant to solve the problem.

Check out `MathUserProxyAgent.message_generator`:
<!-- TODO: Revise this doc accordingly -->
- You may choose from `['default', 'python', 'two_tools']` for parameter `prompt_type`.  We include two more prompts in the paper: 
    1. `'python'` is a simplified prompt from the default prompt that uses Python only. 
    2. `'two_tools'` further allows the selection of Python or Wolfram Alpha based on this simplified `python` prompt. Note that this option requires a Wolfram Alpha API key and put it in `wolfram.txt`.

- You can also input your customized prompt if needed.
Since this mathproxyagent detects '\boxed{}' as termination, you need to have a similar termination sentence in the prompt: "If you get the answer, put the answer in \\boxed{}.". If the customized is provided, the `prompt_type` will be ignored.


### Example 4 (Use the "python" prompt):

Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?

Correct Solution: Subtracting the two equations gives: 
\begin{align*}
(729x+731y)-(725x+727y) &= 1508-1500\\
\Rightarrow\qquad 4x+4y &= 8\\
\Rightarrow\qquad x+y &= 2.
\end{align*}

Multiplying this equation by 725 and subtracting this equation from $725x+727y=1500$ gives \begin{align*}
(725x+727y) - 725(x+y) &= 1500-725(x+y) \implies \\
2y &= 50.
\end{align*}So we can write $x-y$ as $(x+y) - 2y$, which equals  $2 - 50 = \boxed{-48}$.


In [6]:
# we set the prompt_type to "python", which is a simplified version of the default prompt.
math_problem = "Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?"
mathproxyagent.initiate_chat(
    assistant, message=mathproxyagent.message_generator, problem=math_problem, prompt_type="python"
)

mathproxyagent (to assistant):

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \boxed{}.

Problem: Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?

--------------------------------------------------------------------------------
assistant (to mathpr

ChatResult(chat_id=None, chat_history=[{'content': "Let's use Python to solve a math problem.\n\nQuery requirements:\nYou should always use the 'print' function for the output and use fractions/radical forms instead of decimals.\nYou can use packages like sympy to help you.\nYou must follow the formats below to write your code:\n```python\n# your code\n```\n\nPlease follow this process:\n1. Solve the problem step by step (do not over-divide the steps).\n2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).\n3. Wait for me to give the results.\n4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.\n\nAfter all the queries are run and you get the answer, put the answer in \\boxed{}.\n\nProblem: Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?", 'role': 'assistant'}, {'content': "To start solving the syste